# OG SBF Analyze
Version 28 Feb 2025, J. Jensen

Run elliprof, measure mbar, and get uncertainties due to sky and PSF fitting
assuming the masks, SE, and Dophot files are in place and elliprof is optimized


In [1]:
### Install the required python packages
import sys, os
import datetime

pysbf_path = "/Users/Joe/data/sbf/"
config_path = "/Users/Joe/data/sbf/pysbf/config/"
sys.path.insert(0, pysbf_path)
from pysbf import *
from astropy.time import Time

import astropy.io.fits as fits
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pylab as py
import json
import re

configFolder = pysbf_path + "pysbf/config/sextractor/"

version = "SBFsetup: 2025-02-24"

## Object Initialization

In [2]:
now = datetime.now()
SBF_root = '/Users/Joe/data/wfc3-17436/'
referencepsf = '/Users/Joe/data/wfc3-17446/psflibrary/combinedj.on.psf.17446'
overwrite = False

# JWST TRGB 3055 calibrators
# SBF_root = '/Users/Joe/data/wfc3-xxxxx'
# NR=str(150)
#name = "n4374" 
#name = "n4406"
#name = "n4621"
#name = "n4486"
#-name = "n4552"
#-name = "n4697" 
#name = "n1549"
#name = "n3379"

## MASSIVE and SN sample for future re-reduction 
# SBF_root = '/Users/Joe/data/wfc3-xxxxx'
# name = "n4874"
# name = "n4993"
# name = "ic2597"
# name = "n0057"
# name = "n0315"
# name = "n0383"
# name = "n0410"
# name = "n0495"
# name = "n0507"
# name = "n0524"
# name = "n0533"
# name = "n0545"
# name = "n0547"
# name = "n0665"
# name = "n0708"
# name = "n0741"
# name = "n0777"
# name = "n0809"
# name = "n0890"
# name = "n0910"
# name = "n1015"
# name = "n1016"
# name = "n1060"
# name = "n1129"
# name = "n1167"
# name = "n1200"
# name = "n1201"
# name = "n1259"
# name = "n1272"
# name = "n1278"
# name = "n1453"
# name = "n1573"
# name = "n1600"
# name = "n1684"
# name = "n1700"
# name = "n2258"
# name = "n2274"
# name = "n2340"
# name = "n2513"
# name = "n2672"
# name = "n2693"
# name = "n2765"
# name = "n2962"
# name = "n3158"
# name = "n3392"
# name = "n3842"
# name = "n4036"
# name = "n4073"
# name = "n4386"
# name = "n4839"
# name = "n4914"
# name = "n5322"
# name = "n5353"
# name = "n5490"
# name = "n5557"
# name = "n5839"
# name = "n6482"
# name = "n6702"
# name = "n6964"
# name = "n7052"
# name = "n7242"
# name = "n7619"

# GO-16262 SNAP28 Galaxies 
# SBF_root = '/Users/Joe/data/wfc3-16262/'
# name = "cgcg-328-014"
# name = "cgcg-539-126"
# name = "cgcg-540-079"
# name = "eso436-g045"
# name = "eso461-g007"
# name = "eso462-g015"
# name = "eso507-g025"
# name = "ic5193"
# name = "n0080"
# name = "n0380"
# name = "n0679"
# name = "n0750"
# name = "n2208"
# name = "n2256"
# name = "n2329"
# name = "n2418"
# name = "n2569"
# name = "n3070"
# name = "n3091"
# name = "n3308"
# name = "n3311"
# name = "n4825"
# name = "n4955"
# name = "n6223"
# name = "n6577"
# name = "n6688"
# name = "n6968"
# name = "n7265"
# name = "n7274"
# name = "n7426"
# name = "n7618"
# name = "pgc158229"
# name = "pgc170207"
# name = "u03353"
# name = "u03396"
# name = "u11990"
# name = "u12517"

# GO=17446 SN31
# SBF_root = '/Users/Joe/data/wfc3-17446/'
##name = "cgcg-005-038"; sky=2890;skysig=50;r0_1='3';r1_1='200';nr_1='15';r0_2='5';r1_2='220';nr_2='15';r0_3='4';r1_3='240';nr_3='15';nfit='15';ks0='50';ks1='80'; flavor="doj"
##name = "cgcg-031-049"; sky=3025;skysig=50;r0_1='20';r1_1='300';nr_1='23';r0_2='22';r1_2='330';nr_2='25';r0_3='20';r1_3='320';nr_3='27';nfit='10';ks0='40';ks1='80'; flavor="doj"
##name = "eso352-g057"; sky=4500;skysig=100;r0_1='3';r1_1='480';nr_1='15';r0_2='5';r1_2='500';nr_2='18';r0_3='4';r1_3='500';nr_3='17';nfit='15';ks0='40';ks1='80'; flavor="doj"
##name = "eso442-g015"; sky=3500;skysig=200;r0_1='10';r1_1='320';nr_1='12';r0_2='9';r1_2='320';nr_2='10';r0_3='8';r1_3='330';nr_3='12';nfit='15';ks0='50';ks1='80'; flavor="doj"
##name = "eso479-g007"; sky=3500;skysig=200;r0_1='7';r1_1='250';nr_1='25';r0_2='10';r1_2='250';nr_2='30';r0_3='8';r1_3='270';nr_3='25';nfit='10';ks0='40';ks1='80';flavor="sej"
##name = "ic0511"; sky=3760;skysig=100;r0_1='13';r1_1='430';nr_1='8';r0_2='10';r1_2='400';nr_2='8';r0_3='12';r1_3='420';nr_3='8';nfit='15';ks0='40';ks1='80'; flavor="sej"
#-name = "mcg-02-33-017"; sky=5750;skysig=100;r0_1='3';r1_1='530';nr_1='25';r0_2='5';r1_2='510';nr_2='30';r0_3='4';r1_3='550';nr_3='25';nfit='10';ks0='70';ks1='100'; flavor="doj"
#-name = "cgcg-097-050"; sky=4950;skysig=100;r0_1='3';r1_1='390';nr_1='25';r0_2='5';r1_2='400';nr_2='30';r0_3='4';r1_3='';nr_3='410';nfit='10';ks0='30';ks1='80'; flavor="doj"
#-name = "mcg+08-07-008"; sky=5750;skysig=100;r0_1='3';r1_1='530';nr_1='25';r0_2='5';r1_2='520';nr_2='23';r0_3='4';r1_3='510';nr_3='25';nfit='10';ks0='50';ks1='90'; flavor="doj"
#-name = "u0402"; sky=14200;skysig=500;r0_1='3';r1_1='330';nr_1='15';r0_2='5';r1_2='340';nr_2='14';r0_3='7';r1_3='340';nr_3='13';nfit='15';ks0='50';ks1='90'; flavor="doj"
##name = "n0083"; sky=9900;skysig=100;r0_1='3';r1_1='400';nr_1='20';r0_2='5';r1_2='430';nr_2='20';r0_3='4';r1_3='420';nr_3='20';nfit='10';ks0='60';ks1='90'; flavor="doj"
##name = "n1209"; sky=6200;skysig=200;r0_1='3';r1_1='630';nr_1='25';r0_2='5';r1_2='630';nr_2='30';r0_3='4';r1_3='650';nr_3='25';nfit='10';ks0='40';ks1='80'; flavor="doj"
##name = "n3332"; sky=4300;skysig=100;r0_1='3';r1_1='520';nr_1='25';r0_2='5';r1_2='560';nr_2='30';r0_3='4';r1_3='540';nr_3='25';nfit='10';ks0='50';ks1='90'; flavor="doj"
##name = "n3643"; sky=5200;skysig=50;r0_1='15';r1_1='350';nr_1='20';r0_2='18';r1_2='350';nr_2='23';r0_3='12';r1_3='380';nr_3='25';nfit='10';ks0='30';ks1='90'; flavor="doj"
##name = "n3941"; sky=4000;skysig=500;r0_1='3';r1_1='630';nr_1='25';r0_2='5';r1_2='610';nr_2='30';r0_3='4';r1_3='650';nr_3='25';nfit='10';ks0='30';ks1='80'; flavor="doj"
##name = "n4125"; sky=4000; skysig=500; r0_1='3'; r1_1='630'; nr_1='25'; r0_2='5'; r1_2='610'; nr_2='30'; r0_3='4'; r1_3='650'; nr_3='25'; nfit='10'; ks0='50'; ks1='90'; flavor="doj"
#-name = "n4169"; sky=5300; skysig=100; r0_1='3'; r1_1='580'; nr_1='25'; r0_2='5'; r1_2='610'; nr_2='30'; r0_3='4'; r1_3='600'; nr_3='28'; nfit='10'; ks0='40'; ks1='80'; flavor="doj"
#-name = "n4415"; sky=5200; skysig=100; r0_1='3'; r1_1='530'; nr_1='25'; r0_2='5'; r1_2='530'; nr_2='22'; r0_3='4'; r1_3='550'; nr_3='25'; nfit='10'; ks0='30'; ks1='80'; flavor="doj"
##name = "n4636"; sky=12750;skysig=500;r0_1='3';r1_1='530';nr_1='25';r0_2='5';r1_2='510';nr_2='30';r0_3='4';r1_3='550';nr_3='25';nfit='10';ks0='30';ks1='80'; flavor="doj"
##name = "n4767"; sky=3900;skysig=200;r0_1='6';r1_1='630';nr_1='25';r0_2='5';r1_2='620';nr_2='30';r0_3='4';r1_3='650';nr_3='25';nfit='10';ks0='40';ks1='80'; flavor="doj"
##name = "n5018"; sky=5200; skysig=200; r0_1='3'; r1_1='550'; nr_1='25'; r0_2='5'; r1_2='560'; nr_2='30'; r0_3='4'; r1_3='600'; nr_3='25'; nfit='12'; ks0='50'; ks1='80'; flavor="doj"
##name = "n5222"; sky=10600;skysig=200;r0_1='10';r1_1='510';nr_1='17';r0_2='14';r1_2='510';nr_2='15';r0_3='12';r1_3='540';nr_3='17';nfit='15';ks0='50';ks1='80';flavor="sej"
##name = "n5304"; sky=5550; skysig=150; r0_1='8'; r1_1='580'; nr_1='25'; r0_2='10'; r1_2='550'; nr_2='30'; r0_3='9'; r1_3='550'; nr_3='25'; nfit='12'; ks0='40'; ks1='80'; flavor="doj"
#-name = "n5419"; sky=7900;skysig=200;r0_1='3';r1_1='580';nr_1='25';r0_2='5';r1_2='560';nr_2='30';r0_3='4';r1_3='600';nr_3='25';nfit='10';ks0='40';ks1='80'; flavor="doj"
#-name = "n5631"; sky=3600;skysig=100;r0_1='3';r1_1='530';nr_1='25';r0_2='5';r1_2='510';nr_2='30';r0_3='4';r1_3='550';nr_3='25';nfit='10';ks0='35';ks1='65'; flavor="doj"
##name = "n7187"; sky=4400;skysig=50;r0_1='14';r1_1='250';nr_1='11';r0_2='14';r1_2='260';nr_2='10';r0_3='14';r1_3='280';nr_3='10';nfit='15';ks0='50';ks1='90'; flavor="doj"
##name = "leda1693718"; sky=5870;skysig=200;r0_1='3';r1_1='230';nr_1='12';r0_2='3';r1_2='220';nr_2='10';r0_3='4';r1_3='240';nr_3='13';nfit='12';ks0='50';ks1='90'; flavor="doj"
##name = "u2829"; sky=17200;skysig=300;r0_1='12';r1_1='500';nr_1='18';r0_2='10';r1_2='480';nr_2='15';r0_3='12';r1_3='490';nr_3='18';nfit='15';ks0='50';ks1='100'; flavor="doj" # this is a two-orbit exposure.

#GO=17436 SNAP31 
SBF_root = '/Users/Joe/data/wfc3-17436/'
name = "n0050"; sky=3340;skysig=50;r0_1='3';r1_1='500';nr_1='25';r0_2='5';r1_2='480';nr_2='30';r0_3='4';r1_3='510';nr_3='25';nfit='10';ks0='50';ks1='80';flavor='doj'
# name = "n0194"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "n0193"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "n0227"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "n0311"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "n0393"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "n0499"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "n0508"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "n0529"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "n0541"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "n0564"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#-name = "n0759"; sky=3700;skysig=50;r0_1='7';r1_1='530';nr_1='25';r0_2='8';r1_2='510';nr_2='30';r0_3='7';r1_3='550';nr_3='25';nfit='10';ks0='50';ks1='80'; flavor="doj"
# name = "n0883"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "u01859"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "n1057"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
# name = "n1066"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
##name = "ic0265"; sky=3225;skysig=25;r0_1='4';r1_1='370';nr_1='20';r0_2='5';r1_2='370';nr_2='25';r0_3='4';r1_3='380';nr_3='23';nfit='10';ks0='40';ks1='80';flavor='doj'
# name = "ic0310"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n1270"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n1550"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n1609"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n1682"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n1710"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "u03215" ; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n1713"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "u03683"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n2563"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "ic2437"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n4782"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n5357"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "u00902"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n0997"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n1226"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n1322"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
##name = "2MASX-J04002709+3854173"
#name = "n1497"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "2MASX-J04194579+3530344"
#name = "u03021"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n2340"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#-name = "u03725"; sky=6500;skysig=50;r0_1='3';r1_1='530';nr_1='25';r0_2='5';r1_2='510';nr_2='30';r0_3='4';r1_3='550';nr_3='25';nfit='10';ks0='40';ks1='80'; flavor="doj"
#name = "mcg-01-23-019"; sky=;skysig=;r0_1='';r1_1='';nr_1='';r0_2='';r1_2='';nr_2='';r0_3='';r1_3='';nr_3='';nfit='10';ks0='';ks1='';flavor='doj'
#name = "n3222"
#name = "n3771"
#name = "n3172"
#name = "ic1153"
#name = "n6375"
#name = "n6442"
#name = "ic5180"
#name = "n7315"
#name = "u12179"
#name = "cgcg551-015"
#name = "n3343"
#name = "ic0642"
#name = "n0071"
#name = "ic1143"
#name = "u10918"

In [3]:
# Name conventions for files:
datafile = SBF_root+'/'+name+'/'+name+'_data.json'
jfits = SBF_root+'/'+name+'/'+name+'j.fits'
jresid = SBF_root+'/'+name+'/'+name+'j.resid'
jfiterp = SBF_root+'/'+name+'/'+name+'j.fiterpolate'
jmodel = SBF_root+'/'+name+'/'+name+'j.prf'
dmask = SBF_root+'/'+name+'/'+name+'j.dmask'
mask1 = SBF_root+'/'+name+'/'+name+'j.mask1'
mask2 = SBF_root+'/'+name+'/'+name+'j.mask2'
calibrate = SBF_root+'/'+name+'/calibrate.dat'
centers = SBF_root+'/'+name+'/centers.dat'
flucin = SBF_root+'/'+name+'/'+name+'j.flucin'
profile = SBF_root+'/'+name+'/'+name+'.prf'
dpar = SBF_root+'/'+name+'/'+name+'j.dpar'
inpar = SBF_root+'/'+name+'/'+name+'j.inpar'
skydata = SBF_root+'/'+name+'/skydata/'
tmpdir = SBF_root+'/'+name+'/tmp/'
psflibrary = SBF_root+'/psflibrary/'

In [4]:
# Create PSF library list
# the ones from the 2021 paper:
psfs1 = [psflibrary+'combinedj.bright.psf.14219',
        psflibrary+'combinedj.on.bright.psf.14219',
        psflibrary+'n0315j.psf',
        psflibrary+'n0383j.psf',
        psflibrary+'n0665j.psf',
        psflibrary+'n2672j.psf',
        psflibrary+'n1167j.psf1',
        psflibrary+'n1167j.psf2',
        psflibrary+'n0708j.psf1',
        psflibrary+'n0708j.psf2',
       ]

# the ones from 17446
psfs2 = [psflibrary+'combinedj.bright.psf.14219',
         psflibrary+'combinedj.on.bright.psf.14219',
         psflibrary+'combined.psf.17446',
         psflibrary+'combinedj.on.psf.17446',
         psflibrary+'n5304j.psf1',
         psflibrary+'n5304j.psf2',
         psflibrary+'n5304j.psf3',
         psflibrary+'n5304j.psf4',
         psflibrary+'n5304j.psf5',
         psflibrary+'cgcg-005-038j.psf1',
         psflibrary+'cgcg-031-049j.psf1',
         psflibrary+'cgcg-031-049j.psf2',
         psflibrary+'cgcg-031-049j.psf3',
         psflibrary+'cgcg-097-050j.psf1',
         psflibrary+'cgcg-097-050j.psf2',
         psflibrary+'eso352-g057j.psf1',
         psflibrary+'eso442-g015j.psf1',
         psflibrary+'ic0511j.psf1',
         psflibrary+'ic0511j.psf2',
         psflibrary+'ic0511j.psf3',
         psflibrary+'leda1693718j.psf1',
         psflibrary+'leda1693718j.psf2',
         psflibrary+'leda1693718j.psf3',
         psflibrary+'leda1693718j.psf4',
         psflibrary+'leda1693718j.psf5',
         psflibrary+'mcg+08-07-008j.psf1',
         psflibrary+'mcg+08-07-008j.psf2',
         psflibrary+'mcg+08-07-008j.psf3',
         psflibrary+'n3332j.psf2',
         psflibrary+'n3332j.psf3',
         psflibrary+'n4125j.psf1',
         psflibrary+'n4767j.psf1',
         psflibrary+'n4767j.psf3',
         psflibrary+'n4767j.psf4',
         psflibrary+'n5222j.psf1',
         psflibrary+'n5419j.psf1',
         psflibrary+'u0402j.psf1',
       ]

# centered ones only from 17446
psfs3 = [psflibrary+'combinedj.on.bright.psf.14219',
         psflibrary+'combinedj.on.psf.17446',
         psflibrary+'cgcg-097-050j.psf1',
         psflibrary+'eso352-g057j.psf1',
         psflibrary+'ic0511j.psf1',
         psflibrary+'ic0511j.psf2',
         psflibrary+'leda1693718j.psf2',
         psflibrary+'leda1693718j.psf3',
         psflibrary+'mcg+08-07-008j.psf1',
         psflibrary+'mcg+08-07-008j.psf2',
         psflibrary+'mcg+08-07-008j.psf3',
         psflibrary+'n3332j.psf2',
         psflibrary+'n4767j.psf3',
         psflibrary+'n4767j.psf4',
         psflibrary+'n5222j.psf1',
         psflibrary+'u0402j.psf1',
       ]

# the combined ones only
psfs4 = [psflibrary+'combinedj.bright.psf.14219',
        psflibrary+'combinedj.on.bright.psf.14219',
        psflibrary+'combinedj.on.psf.17446',
        psflibrary+'combinedj.psf.17446',
        ]

# pick your list here:
psfs = psfs3
for psf in psfs:
    print(psf)

/Users/Joe/data/wfc3-17436//psflibrary/combinedj.on.bright.psf.14219
/Users/Joe/data/wfc3-17436//psflibrary/combinedj.on.psf.17446
/Users/Joe/data/wfc3-17436//psflibrary/cgcg-097-050j.psf1
/Users/Joe/data/wfc3-17436//psflibrary/eso352-g057j.psf1
/Users/Joe/data/wfc3-17436//psflibrary/ic0511j.psf1
/Users/Joe/data/wfc3-17436//psflibrary/ic0511j.psf2
/Users/Joe/data/wfc3-17436//psflibrary/leda1693718j.psf2
/Users/Joe/data/wfc3-17436//psflibrary/leda1693718j.psf3
/Users/Joe/data/wfc3-17436//psflibrary/mcg+08-07-008j.psf1
/Users/Joe/data/wfc3-17436//psflibrary/mcg+08-07-008j.psf2
/Users/Joe/data/wfc3-17436//psflibrary/mcg+08-07-008j.psf3
/Users/Joe/data/wfc3-17436//psflibrary/n3332j.psf2
/Users/Joe/data/wfc3-17436//psflibrary/n4767j.psf3
/Users/Joe/data/wfc3-17436//psflibrary/n4767j.psf4
/Users/Joe/data/wfc3-17436//psflibrary/n5222j.psf1
/Users/Joe/data/wfc3-17436//psflibrary/u0402j.psf1


### Create model and residual images

In [5]:
if not os.path.exists(tmpdir):
    cmd = 'mkdir '+tmpdir
    xcmd(cmd, False)

cmd = 'cp '+SBF_root+'/'+name+'/calibrate.dat '+tmpdir
xcmd(cmd)


''

In [6]:
def runelliprof(name,x0,y0,skyval,r0_1,r1_1,nr_1,r0_2,r1_2,nr_2,r0_3,r1_3,nr_3,jfits,mask1,mask2,dmask,tmpdir):
    """create the elliprof models and residual image for a particular background value
    """
    print('sky value = ',skyval)
    if not os.path.exists(jfits):
        print(jfits, 'does not exist!')
    if not os.path.exists(mask1):
        print(mask1, 'does not exist!')
    if not os.path.exists(mask2):
        print(mask2, 'does not exist!')
    if not os.path.exists(dmask):
        print(dmask, 'does not exists!')
    
    monsta_script = """
        rd 1 """+jfits+"""
        sc 1 """+skyval+"""
        rd 5 """+mask1+"""
        cop 2 1
        mi 2 5
        cop 3 2
        cop 4 2
        elliprof 2 x0="""+x0+""" y0="""+y0+""" r0="""+r0_1+""" r1="""+r1_1+""" nr="""+nr_1+""" niter=10 model
        elliprof 3 x0="""+x0+""" y0="""+y0+""" r0="""+r0_2+""" r1="""+r1_2+""" nr="""+nr_2+""" niter=10 model
        elliprof 4 x0="""+x0+""" y0="""+y0+""" r0="""+r0_3+""" r1="""+r1_3+""" nr="""+nr_3+""" niter=10 model
        ai 2 3
        ai 2 4
        dc 2 3.
        cop 3 1
        si 3 2
        rd 5 """+mask2+"""
        mi 3 5
        cop 4 3
        fiterpolate 4 nx="""+nfit+""" ny="""+nfit+"""
        cop 3 1
        si 3 2
        si 3 4
        rd 5 """+dmask+"""
        mi 3 5
        wd 2 """+tmpdir+name+"""_"""+str(skyval)+"""j.prf
        wd 3 """+tmpdir+name+"""_"""+str(skyval)+"""j.resid
        wd 4 """+tmpdir+name+"""_"""+str(skyval)+"""j.fiterpolate

    """
    if not os.path.exists(jfits):
        print("No galaxy fits file exists. Please construct one or copy from rawdata directory.")
    else:
        run_monsta(monsta_script, 'monsta.pro', 'monsta.log')
    return

In [7]:
def runsbf(name,version,prf,resid,mask,x0,y0,ks0,ks1,psf,psforder,psfk0,psfk1,skyval,tmpdir):
    """measure SBF magnitudes in annuli using a particular PSF and fit ranges
        version is doj or sej for Dophot/SExtractor versions
        skyval can be the sky value or a PSF index
        ks0 and ks1 are the power spectrum fit upper and lower k
        psfk0 and psfk1 are the same for the PSF.
    """
    if not os.path.exists(psf):
        print(psf, 'does not exist!')
    if not os.path.exists(prf):
        print(prf, 'does not exist!')
    if not os.path.exists(resid):
        print(resid, 'does not exist!')
    if not os.path.exists(mask):
        print(mask, 'doe not exist!')

    monsta_script = """
        rd 1 """+psf+"""
        rd 2 """+prf+"""
        rd 3 """+resid+""" 
        rd 5 """+mask+""" 
        mi 3 5

        cop 4 1
        open 6 nr=512 nc=512
        fluc 6 5 mask x0="""+x0+""" y0="""+y0+""" r0=32 r1=64 a0=0 a1=360 ! c0
        abx 6 all mean=mean silent
        if mean>0
            cop 7 6
            mi 7 3
            tv 7
            fluc 6 2 window
            fluc 6 4 expect order="""+psforder+""" k0="""+psfk0+""" k1="""+psfk1+"""
            fft 8 7
            power 4 8
            fluc 6 4 fit ks0="""+ks0+""" ks1="""+ks1+""" ! plot
            print fluc file="""+tmpdir+name+"""_"""+skyval+version+""".c0
        end_if

        cop 4 1
        open 6 nr=512 nc=512
        fluc 6 5 mask x0="""+x0+""" y0="""+y0+""" r0=64 r1=128 a0=0 a1=360 ! c1
        abx 6 all mean=mean silent
        if mean>0
            cop 7 6
            mi 7 3
            tv 7
            fluc 6 2 window
            fluc 6 4 expect order="""+psforder+""" k0="""+psfk0+""" k1="""+psfk1+"""
            fft 8 7
            power 4 8
            fluc 6 4 fit ks0="""+ks0+""" ks1="""+ks1+""" ! plot
            print fluc file="""+tmpdir+name+"""_"""+skyval+version+""".c1
        end_if

        cop 4 1
        open 6 nr=512 nc=512
        fluc 6 5 mask x0="""+x0+""" y0="""+y0+""" r0=128 r1=256 a0=0 a1=360 ! c2
        abx 6 all mean=mean silent
        if mean>0
            cop 7 6
            mi 7 3
            tv 7
            fluc 6 2 window
            fluc 6 4 expect order="""+psforder+""" k0="""+psfk0+""" k1="""+psfk1+"""
            fft 8 7
            power 4 8
            fluc 6 4 fit ks0="""+ks0+""" ks1="""+ks1+""" ! plot
            print fluc file="""+tmpdir+name+"""_"""+skyval+version+""".c2
        end_if
        
        cop 4 1
        open 6 nr=512 nc=512
        fluc 6 5 mask x0="""+x0+""" y0="""+y0+""" r0=256 r1=512 a0=0 a1=90 ! nw
        cop 7 6
        mi 7 3
        tv 7
        fluc 6 2 window
        fluc 6 4 expect order="""+psforder+""" k0="""+psfk0+""" k1="""+psfk1+"""
        fft 8 7
        power 4 8
        fluc 6 4 fit ks0="""+ks0+""" ks1="""+ks1+""" ! plot
        print fluc file="""+tmpdir+name+"""_"""+skyval+version+""".nw

        cop 4 1
        open 6 nr=512 nc=512
        fluc 6 5 mask x0="""+x0+""" y0="""+y0+""" r0=256 r1=512 a0=90 a1=180 ! sw
        cop 7 6
        mi 7 3
        tv 7
        fluc 6 2 window
        fluc 6 4 expect order="""+psforder+""" k0="""+psfk0+""" k1="""+psfk1+"""
        fft 8 7
        power 4 8
        fluc 6 4 fit ks0="""+ks0+""" ks1="""+ks1+""" ! plot
        print fluc file="""+tmpdir+name+"""_"""+skyval+version+""".sw

        cop 4 1
        open 6 nr=512 nc=512
        fluc 6 5 mask x0="""+x0+""" y0="""+y0+""" r0=256 r1=512 a0=180 a1=270 ! se
        cop 7 6
        mi 7 3
        tv 7
        fluc 6 2 window
        fluc 6 4 expect order="""+psforder+""" k0="""+psfk0+""" k1="""+psfk1+"""
        fft 8 7
        power 4 8
        fluc 6 4 fit ks0="""+ks0+""" ks1="""+ks1+""" ! plot
        print fluc file="""+tmpdir+name+"""_"""+skyval+version+""".se

        cop 4 1
        open 6 nr=512 nc=512
        fluc 6 5 mask x0="""+x0+""" y0="""+y0+""" r0=256 r1=512 a270=0 a1=360 ! ne
        cop 7 6
        mi 7 3
        tv 7
        fluc 6 2 window
        fluc 6 4 expect order="""+psforder+""" k0="""+psfk0+""" k1="""+psfk1+"""
        fft 8 7
        power 4 8
        fluc 6 4 fit ks0="""+ks0+""" ks1="""+ks1+""" ! plot
        print fluc file="""+tmpdir+name+"""_"""+skyval+version+""".ne

    """
    if not os.path.exists(jfits):
        print("No galaxy fits file exists. Please construct one or copy from rawdata directory.")
    else:
        run_monsta(monsta_script, 'monsta.pro', 'monsta.log')
    return

In [8]:
def retrieve_mbar_values(mbar_file_path):
    """Retrieve data from traditional MBAR files."""

    with open(mbar_file_path, "r") as f:
        lines = f.readlines()
                
    data1 = []
    columns1 = ["Annulus","<1>","r_arcsec", "<mu>", "DPcut", "phcut", "magdiff", "m0", "m0_err", "P1", "m_r", "m_r_err", "m_g", "m_f", "m_f_err", "mbar", "mbar_err"]
    for line in lines[4:12]:  # Skip first four lines (headers)
        values = line.split()
        if isinstance(values,list) and len(values) == len(columns1):  # Ensure correct number of columns
            data1.append(values)
        else:
            print("Could not parse one of the lines in ",mbar_file_path)
    
    df1 = pd.DataFrame(data1, columns=columns1)
    
    for col in df1.columns[1:]:
        df1[col] = pd.to_numeric(df1[col], errors="coerce")
        
    return df1

In [9]:
# import json file to get centers x0, y0
x0=0; y0=0
with open(datafile, "r") as f:
    data = json.load(f)
    x0 = str(data["Center x"])
    y0 = str(data["Center y"])
    print(x0,y0)

562 560


In [10]:
skyplus = str(sky + skysig)
skyminus = str(sky - skysig)
sky = str(sky)

In [11]:
# Run elliprof for sky +/- sigma and get mbar results

runelliprof(name,x0,y0,sky,r0_1,r1_1,nr_1,r0_2,r1_2,nr_2,r0_3,r1_3,nr_3,jfits,mask1,mask2,dmask,tmpdir)
runelliprof(name,x0,y0,skyplus,r0_1,r1_1,nr_1,r0_2,r1_2,nr_2,r0_3,r1_3,nr_3,jfits,mask1,mask2,dmask,tmpdir)
runelliprof(name,x0,y0,skyminus,r0_1,r1_1,nr_1,r0_2,r1_2,nr_2,r0_3,r1_3,nr_3,jfits,mask1,mask2,dmask,tmpdir)

psf = referencepsf
psforder = '4'
psfk0 = '0'; psfk1 = '20'

oldpwd = os.getcwd()
os.chdir(tmpdir)

version = flavor
prf = tmpdir+name+'_'+skyplus+'j.prf'
resid = tmpdir+name+'_'+skyplus+'j.resid'
mask = SBF_root+'/'+name+'/'+name+version+'.ptm6b'
runsbf(name,version,prf,resid,mask,x0,y0,ks0,ks1,psf,psforder,psfk0,psfk1,skyplus,tmpdir)
cmd = 'bestfluc '+tmpdir+name+'_'+skyplus+version
xcmd(cmd, False)

version = flavor
prf = tmpdir+name+'_'+skyminus+'j.prf'
resid = tmpdir+name+'_'+skyminus+'j.resid'
mask = SBF_root+'/'+name+'/'+name+version+'.ptm6b'
runsbf(name,version,prf,resid,mask,x0,y0,ks0,ks1,psf,psforder,psfk0,psfk1,skyminus,tmpdir)
cmd = 'bestfluc '+tmpdir+name+'_'+skyminus+version
xcmd(cmd, False)

mbarfile = tmpdir+name+'_'+skyplus+flavor+'.MBAR'
skyplus = retrieve_mbar_values(mbarfile)
mbarfile = tmpdir+name+'_'+skyminus+flavor+'.MBAR'
skyminus = retrieve_mbar_values(mbarfile)
skysigma = np.abs((skyplus["mbar"] - skyminus["mbar"]) / 2.)


sky value =  3340


sky value =  3390


sky value =  3290


In [12]:
# Run SBF fit with various PSFs
psforder = '4'
psfk0 = '0'; psfk1 = '20'
version = flavor
prf = jmodel
resid = jresid
mask = SBF_root+'/'+name+'/'+name+version+'.ptm6b'
psfmbars = {"mbarc0": [], "mbarc1": [], "mbarc2": [], "mbarc5": []}

for index, psf in enumerate(psfs, start=1):
    if os.path.exists(psf):
        runsbf(name,version,prf,resid,mask,x0,y0,ks0,ks1,psf,psforder,psfk0,psfk1,str(index),tmpdir)
        mbarfile = tmpdir+name+'_'+str(index)+version
        cmd = 'bestfluc '+mbarfile
        xcmd(cmd, False)
        psfsigma = retrieve_mbar_values(mbarfile+'.MBAR')
        for i in psfsigma.index:
            if i == 0:
                psfmbars["mbarc0"].append(psfsigma.at[0, "mbar"])
            elif i == 1:
                psfmbars["mbarc1"].append(psfsigma.at[1, "mbar"])
            elif i == 2:
                psfmbars["mbarc2"].append(psfsigma.at[2, "mbar"])
            elif i == 7:
                psfmbars["mbarc5"].append(psfsigma.at[7, "mbar"])
    else:
        print(psf, 'does not exist. Continuing.')
        
#print(psfmbars)

In [13]:
# calculate sample standard deviation
if (index > 3):
    psfstdev = {"psfsigc0": [], "psfsigc1": [], "psfsigc2": [], "psfsigc5": []}
    psfstdev["psfsigc0"] = np.std(psfmbars["mbarc0"], ddof=1)
    psfstdev["psfsigc1"] = np.std(psfmbars["mbarc1"], ddof=1)
    psfstdev["psfsigc2"] = np.std(psfmbars["mbarc2"], ddof=1)
    psfstdev["psfsigc5"] = np.std(psfmbars["mbarc5"], ddof=1)
    print(psfstdev)


{'psfsigc0': 0.036878177829171306, 'psfsigc1': 0.039072582032247005, 'psfsigc2': 0.03913651321549534, 'psfsigc5': 0.03896579696776817}


In [14]:
now = datetime.now()
print(now)

2025-03-05 01:16:09.514669


In [15]:
with open(datafile, "r") as f:
    data = json.load(f)

skyuncertainty = []
for i in skysigma.index:
    if i == 0:
        skyuncertainty.append({"Sky uncertainty c0": np.round(skysigma[0],3)})
    elif i == 1:
        skyuncertainty.append({"Sky uncertainty c1": np.round(skysigma[1],3)})
    elif i == 2:
        skyuncertainty.append({"Sky uncertainty c2": np.round(skysigma[2],3)})
    elif i == 7:
        skyuncertainty.append({"Sky uncertainty c5": np.round(skysigma[7],3)})
data["mbar background+elliprof uncertainty"] = skyuncertainty

psfuncertainty = []
psfuncertainty.append({"PSF uncertainty c0": np.round(psfstdev["psfsigc0"],3)})
psfuncertainty.append({"PSF uncertainty c1": np.round(psfstdev["psfsigc1"],3)})
psfuncertainty.append({"PSF uncertainty c2": np.round(psfstdev["psfsigc2"],3)})
psfuncertainty.append({"PSF uncertainty c5": np.round(psfstdev["psfsigc5"],3)})  
data["mbar PSF uncertainty"] = psfuncertainty

with open(datafile, "w") as f:
    json.dump(data, f, indent=4)

print(skyuncertainty)
print(psfuncertainty)

[{'Sky uncertainty c0': 0.0}, {'Sky uncertainty c1': 0.005}, {'Sky uncertainty c2': 0.01}, {'Sky uncertainty c5': 0.02}]
[{'PSF uncertainty c0': 0.037}, {'PSF uncertainty c1': 0.039}, {'PSF uncertainty c2': 0.039}, {'PSF uncertainty c5': 0.039}]


In [16]:
#!cat {datafile}

In [17]:
os.chdir(oldpwd)

In [18]:
# clean up tmp files
cmd = 'rm -r '+tmpdir
xcmd(cmd, False)

''